In [21]:
import pandas as pd
import numpy as np
movief = pd.read_excel('mf_16.xlsx')
#movief = pd.read_excel('mf_64.xlsx')
userf = pd.read_excel('uf_16.xlsx')
#userf = pd.read_excel('uf_64.xlsx')
movies = movief.values
users = userf.values
print(len(movies),len(users))



3882 6040


In [22]:
print(movies)
print(users)

[[2
  '0.25425294,-0.2457244,0.24962181,-0.24581432,-0.24892293,-0.24532984,-0.24586874,-0.25127733,-0.25113833,0.24826722,0.24752617,0.25726625,-0.24414212,-0.25203755,-0.25728127,0.25497824']
 [3
  '0.25017294,-0.2457995,0.25436774,-0.24780421,-0.25060365,-0.24953762,-0.24411681,-0.25705525,-0.24559644,0.24756825,0.25047752,0.2564425,-0.24795435,-0.25041232,-0.25473127,0.24689965']
 [4
  '0.25122598,-0.24346033,0.2533685,-0.2475703,-0.25033432,-0.24930677,-0.2442295,-0.2557094,-0.24711587,0.24805807,0.25064957,0.25729752,-0.24668501,-0.25029847,-0.2556204,0.24859317']
 ...
 [3950
  '0.25064844,-0.24384755,0.2536641,-0.24773964,-0.25032836,-0.24898496,-0.24421787,-0.25552824,-0.24674146,0.24801582,0.2505462,0.2575899,-0.24732833,-0.25045738,-0.2551596,0.24874216']
 [3951
  '0.25142196,-0.24327278,0.25329226,-0.24752927,-0.25018108,-0.24933727,-0.2441545,-0.25574017,-0.24732074,0.24814548,0.25063863,0.2573059,-0.24645485,-0.25029194,-0.25574583,0.24868023']
 [3952
  '0.2507458,-0.24358

In [29]:
movie = []
user = []
for i in range(len(movies)):
    a=movies[i][1].split(',')
    tpl = []
    for j in range(len(a)):
        tpl.append(float(a[j]))
    movie.append(tpl)
for i in range(len(users)):
    a=users[i][1].split(',')
    tpl = []
    for j in range(len(a)):
        tpl.append(float(a[j]))
    user.append(tpl)
print(len(movie),len(user))

3882 6040


In [30]:
import torch
import random
moviet = torch.from_numpy(np.array(movie))
usert = torch.from_numpy(np.array(user))
def int_random(k):
    ls = []
    while len(ls)<k:
        a=random.randint(0,19)
        if(a not in ls):
            ls.append(a)
    return ls

In [71]:
from math import sqrt
def multipl(a,b):
    sumofab = 0.0
    for i in range(len(a)):
        temp = a[i] * b[i]
        sumofab += temp
    return sumofab
def corrcoef(x,y):
    n = len (x)
    sum1 = sum (x)
    sum2 = sum (y)
    sumofxy = multipl(x,y)
    sumofx2 = sum([pow(i,2) for i in x])
    sumofy2 = sum([pow(j,2) for j in y])
    num = sumofxy - (float(sum1)*float(sum2)/n)
    den = sqrt((sumofx2 - float(sum1**2)/n)*(sumofy2 - float(sum2**2)/n))
    return num/den


In [78]:
'''实现三种推荐方式：topK默认为20
1：输入电影ID，推荐相似度最高的5部电影，topK选5
2：输入用户ID，推荐最有可能观看的5部电影，topK选5
3：输入用户ID，推荐相似度最高的5个人分别最有可能看的5部电影，topK选5'''

# return the indexes of movie
# dist function: Euclid distance
def recommend_by_movie(movieIndex,topK=20):
    a = moviet[movieIndex]
    ls1 = []
    for i in range(len(moviet)):
        if (i== movieIndex):
            continue
        ls1.append(torch.dist(moviet[i],a,p=2))
    am = torch.from_numpy(np.array(ls1))
    values, indices = torch.topk(am,topK,largest=False)
    ran = int_random(5)
    recoml = []
    for i in range(len(ran)):
        recoml.append(indices[ran[i]].item())
    return recoml
#print(recommend_by_movie(20))


# return the indexes of movie
# dist function: Pearson correlation
def recommend_by_user_self(userIndex,topK=20):
    a = usert[userIndex]
    ls1 = []
    for i in range(len(moviet)):
        ls1.append(corrcoef(a,moviet[i]))
    am = torch.from_numpy(np.array(ls1))
    values, indices = torch.topk(am,topK,largest=True)
    ran = int_random(5)
    recoml = []
    for i in range(len(ran)):
        recoml.append(indices[ran[i]].item())
    return recoml
#print(recommend_by_user_self(1010))


# return the list of five users and their movies
# dist fumction; Hammming distance
def recommend_by_user_other(userIndex,topK=20):
    a = usert[userIndex]
    ls1 = []
    for i in range(len(usert)):
        if(i==userIndex):
            continue
        ls1.append(torch.dist(usert[i],a,p=1))
    au=torch.from_numpy(np.array(ls1))
    values,indices = torch.topk(au,topK,largest=False)
    ran = int_random(5)
    simuser = []
    for i in range(len(ran)):
        simuser.append(indices[ran[i]].item())
    recoml=[]
    for i in range(len(simuser)):
        recoml.append(recommend_by_user_self(simuser[i]))
    return simuser,recoml
#print(recommend_by_user_other(20))

([5787, 5663, 5646, 3748, 2008], [[717, 1127, 2557, 3072, 674], [3268, 2911, 3072, 2557, 674], [1422, 3123, 2525, 2534, 2200], [1132, 599, 1035, 717, 1091], [3123, 599, 1786, 674, 1132]])
